In [1]:
# !pip install flask
# !pip install --upgrade gevent

In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold


In [20]:
df = pd.read_csv("creditcard.csv")
df['Hours'] = pd.to_datetime(df['Time'],unit='s').dt.strftime('%H')
dataX = df.copy().drop("Class",axis = 1)
dataY = df["Class"].copy()
features_to_scale = dataX.drop(["Time"],axis = 1).columns
sX = StandardScaler()
dataX.loc[:,features_to_scale] = sX.fit_transform(dataX[features_to_scale])
X_train, X_test, y_train, y_test = train_test_split(dataX, dataY, test_size = 0.33, random_state = 2018, stratify = dataY)
k_fold = StratifiedKFold(n_splits=5, random_state= 2018, shuffle= True)

#..........................................Base Model.......................................
#set hyper parameters
penalty = 'l2'
C = 1.0
class_weight = 'balanced'
random_state = 2018
solver = 'liblinear'

logreg = LogisticRegression(penalty= penalty, class_weight= class_weight,
                            random_state= random_state, solver = solver, C=C)
#...........................................................................................
trainingScores = []
cvScores = []
predictionsBasedOnKFolds = pd.DataFrame(data=[],
index=y_train.index,columns=[0,1])
model = logreg

for train_index, cv_index in k_fold.split(np.zeros(len(X_train))
    ,y_train.ravel()):
    
    
    X_train_fold, X_cv_fold = X_train.iloc[train_index,:], \
    X_train.iloc[cv_index,:]
    y_train_fold, y_cv_fold = y_train.iloc[train_index], \
    y_train.iloc[cv_index]
    
    model.fit(X_train_fold, y_train_fold)
    
    loglossTraining = log_loss(y_train_fold,
                            model.predict_proba(X_train_fold)[:,1])
    
    trainingScores.append(loglossTraining)
    
    predictionsBasedOnKFolds.loc[X_cv_fold.index,:] = \
        model.predict_proba(X_cv_fold)
    loglossCV = log_loss(y_cv_fold,
                         predictionsBasedOnKFolds.loc[X_cv_fold.index,1])
    
    cvScores.append(loglossCV)
    print('Training Log Loss: ', loglossTraining)
    print('CV Log Loss: ', loglossCV)
    
loglossLogisticRegression = log_loss(y_train,
predictionsBasedOnKFolds.loc[:,1])
print('Logistic Regression Log Loss: ', loglossLogisticRegression)



Training Log Loss:  0.11025046039889204
CV Log Loss:  0.10848572064347986
Training Log Loss:  0.10352520264994451
CV Log Loss:  0.10242304587867627
Training Log Loss:  0.11536522142475307
CV Log Loss:  0.11885175956282508
Training Log Loss:  0.11527942541807791
CV Log Loss:  0.11806537120578031
Training Log Loss:  0.09603773575352166
CV Log Loss:  0.09695545828580006
Logistic Regression Log Loss:  0.10895627111531231


In [22]:
import pickle
pickle.dump(model , open('base_model.pkl','wb'))

In [23]:
base_model = pickle.load(open('base_model.pkl','rb'))
y_predict = base_model.predict(X_test)
test_data = X_test.copy()
test_data['Prediction'] = y_predict
print(test_data['Prediction'].value_counts())
test_data.head(2)

0    91625
1     2362
Name: Prediction, dtype: int64


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V22,V23,V24,V25,V26,V27,V28,Amount,Hours,Prediction
67353,52490.0,0.600923,-0.971676,0.077752,-0.889221,-1.099568,-0.260433,-0.655304,-0.060183,-1.682563,...,-0.823788,-0.220612,-0.021329,0.699351,-0.550067,-0.016426,0.097134,0.349234,-0.007963,0
67626,52625.0,-1.706842,-2.645103,0.971504,-1.579282,0.183585,-0.114004,0.102960,-0.263008,1.534994,...,0.119220,2.496674,-1.283316,1.380894,0.184575,-1.064427,0.702734,0.676278,-0.007963,0


In [2]:
from flask import Flask, render_template, request, jsonify


In [ ]:
app = Flask(__name__)
# @app.route('/predict',methods=['POST'])
# def predict():
#     features = np.array( [int(x) for x in request.form.values()] )
#     prediction = base_model.predict(features)
#     return render_template('index.html',prediction_in_html = f'{prediction}')




In [5]:

@app.route("/")
def hello(): 
  return "Hello World"
  #return render_template('index.html')
app.run()

# import portpicker
# port = portpicker.pick_unused_port()
# from google.colab import output
# output.serve_kernel_port_as_window(port)

# from gevent.pywsgi import WSGIServer
# host='localhost'
# app_server = WSGIServer((host, port), app)
# app_server.serve_forever()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [25/Nov/2021 22:57:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2021 22:57:01] "GET /favicon.ico HTTP/1.1" 404 -
